In [1]:
import re

import numpy as np
import pandas as pd

from pathlib import Path
from datetime import datetime

In [2]:
# Carregar o dataframe df_acordaos
start_time = datetime.now()
df_acordaos = pd.read_csv('dados/acordaos.csv', sep=',', error_bad_lines=False, warn_bad_lines=False, low_memory=False)
df_acordaos.rename(str.upper, axis='columns', inplace=True)
print('Duration: {}'.format(datetime.now() - start_time))
df_acordaos.info()

Duration: 0:00:38.754526
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309806 entries, 0 to 309805
Data columns (total 28 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   ACORDAO                           288120 non-null  object 
 1   ADVOGADO                          223477 non-null  object 
 2   ANOACORDAO                        309806 non-null  int64  
 3   ASSUNTO                           77524 non-null   object 
 4   COLEGIADO                         309806 non-null  object 
 5   DATASESSAO                        308360 non-null  object 
 6   DECISAO                           19451 non-null   object 
 7   DECLARACAOVOTO                    3328 non-null    object 
 8   ENTIDADE                          253844 non-null  object 
 9   INTERESSADOS                      255674 non-null  object 
 10  MINISTROALEGOUIMPEDIMENTOSESSAO   5185 non-null    object 
 11  MINISTROAPRESENTOUDECLARACA

In [3]:
df_acordaos.drop(['REPRESENTANTEMP', 'UNIDADETECNICA', 'UNIDADETECNICA', 'MINISTROREVISOR',
                  'MINISTROAUTORVOTOVENCEDOR', 'MINISTROAPRESENTOUDECLARACAOVOTO',
                 'MINISTROALEGOUIMPEDIMENTOSESSAO', 'INTERESSADOS', 'RECURSOS', 'ENTIDADE',
                 'DECLARACAOVOTO', 'ADVOGADO', 'RELATORDELIBERACAORECORRIDA'], axis=1, inplace=True)

df_acordaos.rename(columns={'ANOACORDAO':'ANO_ACORDAO', 'NUMACORDAO':'NUMERO_ACORDAO',
                           'TIPOPROCESSO':'TIPO_PROCESSO', 'DATASESSAO':'DATA_SESSAO',
                           'NUMATA':'NUMERO_ATA', 'PROC':'PROCESSO'}, inplace=True)

df_acordaos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309806 entries, 0 to 309805
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ACORDAO              288120 non-null  object
 1   ANO_ACORDAO          309806 non-null  int64 
 2   ASSUNTO              77524 non-null   object
 3   COLEGIADO            309806 non-null  object
 4   DATA_SESSAO          308360 non-null  object
 5   DECISAO              19451 non-null   object
 6   NUMERO_ACORDAO       309806 non-null  object
 7   NUMERO_ATA           307239 non-null  object
 8   PROCESSO             307121 non-null  object
 9   RELATOR              307262 non-null  object
 10  RELATORIO            110737 non-null  object
 11  SUMARIO              115466 non-null  object
 12  TIPO_PROCESSO        298928 non-null  object
 13  VOTO                 110475 non-null  object
 14  VOTOCOMPLEMENTAR     2517 non-null    object
 15  VOTOMINISTROREVISOR  2925 non-null

In [4]:
df_acordaos.insert(0, 'ID', range(1, len(df_acordaos)+1))
# Transfora em string a coluna ID
df_acordaos['ID'] = df_acordaos['ID'].map(str)
df_acordaos['ANO_ACORDAO'] = df_acordaos['ANO_ACORDAO'].map(str)
df_acordaos.insert(13, 'TIPO_DECISAO', np.NaN)

In [5]:
# DE -> DECISAO = 19451 : 647 marcados como SIGILOSO por esse motivo não possui ACORDAO, VOTO, RELATORIO, SUMARIO
df_acordaos.loc[(df_acordaos.DECISAO.notnull() == True) & ((df_acordaos.ACORDAO.isnull()==True) | (df_acordaos.ACORDAO.str.contains('SIGILOSO', na=False))), 'TIPO_DECISAO'] = 'DE'

In [6]:
# AR -> ACORDAO DE RELACAO : 193963
df_acordaos.loc[(df_acordaos.ACORDAO.notnull()) & (df_acordaos.DECISAO.isnull()) & (df_acordaos.RELATORIO.isnull()) & (df_acordaos.VOTO.isnull()), 'TIPO_DECISAO'] = 'AR'

In [7]:
df_acordaos.loc[df_acordaos.TIPO_DECISAO.isna(), 'TIPO_DECISAO'] = 'AC'

In [8]:
df_acordaos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309806 entries, 0 to 309805
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ID                   309806 non-null  object
 1   ACORDAO              288120 non-null  object
 2   ANO_ACORDAO          309806 non-null  object
 3   ASSUNTO              77524 non-null   object
 4   COLEGIADO            309806 non-null  object
 5   DATA_SESSAO          308360 non-null  object
 6   DECISAO              19451 non-null   object
 7   NUMERO_ACORDAO       309806 non-null  object
 8   NUMERO_ATA           307239 non-null  object
 9   PROCESSO             307121 non-null  object
 10  RELATOR              307262 non-null  object
 11  RELATORIO            110737 non-null  object
 12  SUMARIO              115466 non-null  object
 13  TIPO_DECISAO         309806 non-null  object
 14  TIPO_PROCESSO        298928 non-null  object
 15  VOTO                 110475 non-nu

In [9]:
def format_numero_acordao(valor):
    string = re.sub(re.compile('\D'), '', valor)
    if len(string)>4:
        return string[:-4]
    else:
        return string
    
    
def extrairTipoSessao(texto):
    if len(re.findall('PRIMEIRA', texto)) != 0:
        return '1C'
    elif len(re.findall('SEGUNDA', texto)) > 0:
        return '2C'
    else:
        return 'PL'
    
    
def criaChaveAcordao(row):

    return row.TIPO_DECISAO + '-' + row.NUMERO_ACORDAO.zfill(6) + '-' + str(row.ANO_ACORDAO) + '-' + extrairTipoSessao(row.COLEGIADO.upper())

In [10]:
df_acordaos.NUMERO_ACORDAO = df_acordaos.NUMERO_ACORDAO.apply(format_numero_acordao)

In [11]:
start_time = datetime.now()

df_acordaos.ID = df_acordaos.apply(criaChaveAcordao, axis=1)

print('Duration: {}'.format(datetime.now() - start_time))

Duration: 0:00:07.649222


In [12]:
# 2871 registros com os campos, ACORDAO, DECISAO, VOTO e RELATORIO nulos simultaneamente e por isso excluídos
df_acordaos.drop(df_acordaos.loc[(df_acordaos.ACORDAO.isnull()) & df_acordaos.DECISAO.isnull() & df_acordaos.RELATORIO.isnull() & df_acordaos.VOTO.isnull()].index, inplace=True)

In [13]:
# Registros marcados como SILIGOSO e por esse motivo convertidos para NaN a fim de padronização
df_acordaos.loc[(df_acordaos.ACORDAO.str.lower() == 'sigiloso'), 'ACORDAO'] = np.NaN
df_acordaos.loc[(df_acordaos.DECISAO.str.lower() == 'sigiloso'), 'DECISAO'] = np.NaN
df_acordaos.loc[(df_acordaos.RELATORIO.str.lower() == 'sigiloso'), 'RELATORIO'] = np.NaN
df_acordaos.loc[(df_acordaos.VOTO.str.lower() == 'sigiloso'), 'VOTO'] = np.NaN

In [14]:
df_acordaos.fillna('', inplace=True)

In [15]:
df_acordaos['DECISAO'] = df_acordaos.ACORDAO + ' ' + df_acordaos.DECISAO + ' ' + df_acordaos.RELATORIO + ' ' + df_acordaos.VOTO + ' ' + df_acordaos.VOTOCOMPLEMENTAR + ' ' + df_acordaos.VOTOMINISTROREVISOR

In [16]:
# Retira colunas que não interessam para o trabalho
df_acordaos = df_acordaos.drop(['ANO_ACORDAO', 'ASSUNTO', 'COLEGIADO', 'DATA_SESSAO', 'NUMERO_ACORDAO', 
                                'NUMERO_ATA', 'PROCESSO', 'RELATOR', 'SUMARIO', 'TIPO_PROCESSO', 'ACORDAO', 
                                'VOTO', 'RELATORIO', 'VOTOCOMPLEMENTAR', 'VOTOMINISTROREVISOR', 
                                'TIPO_DECISAO'], axis=1)

df_acordaos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306935 entries, 0 to 309804
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   ID       306935 non-null  object
 1   DECISAO  306935 non-null  object
dtypes: object(2)
memory usage: 7.0+ MB


In [17]:
df_acordaos.sample(2)

,ID,DECISAO
15023,AC-000168-1997-1C,"VISTOS, relatados e discutidos estes autos ..."
44659,AC-000333-2003-2C,"VISTOS, relatados e discutidos estes autos de ..."


In [18]:
# df_acordaos.to_csv('dados/df_acordaos.csv', sep='|', index=False)

<h2>Trabalhando com o dataset de acórdãos rotulados

In [19]:
df_rotulados = pd.read_csv('dados/AC_CLASS_IMPORT.csv', sep=';', encoding='iso-8859-1')
df_rotulados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   NUMACORDAO  330 non-null    object 
 1   LABEL_A     330 non-null    int64  
 2   LABEL_B     32 non-null     float64
 3   LABEL_C     3 non-null      float64
dtypes: float64(2), int64(1), object(1)
memory usage: 10.4+ KB


In [20]:
df_rotulados.rename(columns={'NUMACORDAO': 'ID'}, inplace=True)

df_rotulados.fillna(0, inplace=True)

df_rotulados.LABEL_A = df_rotulados.LABEL_A.astype(str)

df_rotulados.LABEL_B = df_rotulados.LABEL_B.astype(int).astype(str)
df_rotulados.loc[df_rotulados.LABEL_B == '0', 'LABEL_B'] = np.NaN

df_rotulados.LABEL_C = df_rotulados.LABEL_C.astype(int).astype(str)
df_rotulados.loc[df_rotulados.LABEL_C == '0', 'LABEL_C'] = np.NaN

df_rotulados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ID       330 non-null    object
 1   LABEL_A  330 non-null    object
 2   LABEL_B  32 non-null     object
 3   LABEL_C  3 non-null      object
dtypes: object(4)
memory usage: 10.4+ KB


In [21]:
df_rotulados

,ID,LABEL_A,LABEL_B,LABEL_C
0,AC-000035-2018-PL,1,2,NaN
1,AC-000043-2008-PL,7,NaN,NaN
2,AC-000103-2009-PL,7,NaN,NaN
3,AC-000107-2009-PL,2,NaN,NaN
4,AC-000111-2016-PL,7,NaN,NaN
...,...,...,...,...
325,DE-000417-2002-PL,2,NaN,NaN
326,DE-000434-1993-PL,5,7,NaN
327,DE-000767-1998-PL,2,NaN,NaN
328,DE-001025-2001-PL,5,NaN,NaN


In [22]:
# df_acordaos = pd.read_csv('dados/df_acordaos.csv', sep='|')

In [23]:
path = 'dados/'
# Seguindo os parâmetros da lib pathlib
type_file = '**/*.txt'

In [24]:
df_acordaos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306935 entries, 0 to 309804
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   ID       306935 non-null  object
 1   DECISAO  306935 non-null  object
dtypes: object(2)
memory usage: 7.0+ MB


In [25]:
for path in Path('dados/').rglob('**/*.txt'):
    texto = ''
    with path.open() as f:
        texto += f.read()
    df_acordaos = df_acordaos.append({'ID': path.stem, 'DECISAO': texto}, ignore_index=True)

In [26]:
df_acordaos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306988 entries, 0 to 306987
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   ID       306988 non-null  object
 1   DECISAO  306988 non-null  object
dtypes: object(2)
memory usage: 4.7+ MB


In [27]:
df_acordaos[df_acordaos.duplicated(['ID', 'DECISAO'])]

,ID,DECISAO
3390,DE-000088-1993-PL,"O Tribunal Pleno, diante das razões expostas ..."
4010,DE-000037-1993-PL,[SIGILOSO] [SIGILOSO]
4011,DE-000036-1993-PL,[SIGILOSO] [SIGILOSO]
4066,DE-000029-1993-PL,[SIGILOSO] [SIGILOSO]
4067,DE-000030-1993-PL,[SIGILOSO] [SIGILOSO]
4081,DE-000026-1993-PL,[SIGILOSO] [SIGILOSO]
4082,DE-000025-1993-PL,[SIGILOSO] [SIGILOSO]
4083,DE-000028-1993-PL,[SIGILOSO] [SIGILOSO]
4168,DE-000018-1993-PL,[SIGILOSO] [SIGILOSO]
4169,DE-000017-1993-PL,[SIGILOSO] [SIGILOSO]


In [28]:
# df_acordaos.drop_duplicates(subset=['ID', 'DECISAO'], keep='first', inplace=True)

In [29]:
df_acordaos[df_acordaos.duplicated(['ID', 'DECISAO'])]

,ID,DECISAO
3390,DE-000088-1993-PL,"O Tribunal Pleno, diante das razões expostas ..."
4010,DE-000037-1993-PL,[SIGILOSO] [SIGILOSO]
4011,DE-000036-1993-PL,[SIGILOSO] [SIGILOSO]
4066,DE-000029-1993-PL,[SIGILOSO] [SIGILOSO]
4067,DE-000030-1993-PL,[SIGILOSO] [SIGILOSO]
4081,DE-000026-1993-PL,[SIGILOSO] [SIGILOSO]
4082,DE-000025-1993-PL,[SIGILOSO] [SIGILOSO]
4083,DE-000028-1993-PL,[SIGILOSO] [SIGILOSO]
4168,DE-000018-1993-PL,[SIGILOSO] [SIGILOSO]
4169,DE-000017-1993-PL,[SIGILOSO] [SIGILOSO]


In [30]:
df_acordaos = pd.merge(df_acordaos, df_rotulados, on='ID', how='left')
df_acordaos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306988 entries, 0 to 306987
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   ID       306988 non-null  object
 1   DECISAO  306988 non-null  object
 2   LABEL_A  328 non-null     object
 3   LABEL_B  32 non-null      object
 4   LABEL_C  3 non-null       object
dtypes: object(5)
memory usage: 14.1+ MB


In [32]:
df_rotulados.LABEL_A.value_counts()

7     152
2      61
5      48
8      22
1      20
10     10
11      6
4       4
9       3
3       2
6       2
Name: LABEL_A, dtype: int64

In [31]:
df_acordaos.LABEL_A.value_counts()

7     150
2      59
5      49
8      22
1      21
10     10
11      6
4       4
9       3
3       2
6       2
Name: LABEL_A, dtype: int64

In [33]:
# Salva o arquivo em um csv
df_acordaos.loc[df_acordaos.LABEL_A.notnull()].to_csv('dados/df_rotulados.csv', sep='|', index=False)

In [34]:
df_acordaos[['ID', 'LABEL_A', 'LABEL_B', 'LABEL_C']].to_csv('dados/temp.csv', sep=';', index=False)